# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [6]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 13
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [ ]:
pip install dlt[duckdb]


In [17]:
import dlt
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


In [11]:
pipeline = dlt.pipeline(
    pipeline_name="persons_loading",
    destination="duckdb",
    dataset_name="data_new"
)



In [18]:
load_info_1 = pipeline.run(people_1, table_name="persons", write_disposition="replace")
print(load_info_1)

Pipeline persons_loading load step completed in 0.32 seconds
1 load package(s) were loaded to destination duckdb and into dataset data
The duckdb destination used duckdb:////content/persons_loading.duckdb location to store data
Load package 1708302103.6040869 is LOADED and contains no failed jobs


In [19]:
load_info_2 = pipeline.run(people_2, table_name="persons", write_disposition="append")
print(load_info_2)

Pipeline persons_loading load step completed in 0.22 seconds
1 load package(s) were loaded to destination duckdb and into dataset data
The duckdb destination used duckdb:////content/persons_loading.duckdb location to store data
Load package 1708302112.782079 is LOADED and contains no failed jobs


In [25]:
import duckdb

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
#print('Loaded Tables:')
#display(conn.sql("show tables"))

persons = conn.sql("SELECT * FROM persons").df()
total = persons['age'].sum()
print(total)

353


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [ ]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [33]:
load_info_1 = pipeline.run(people_1, table_name="new_persons", write_disposition="replace")
load_info_2 = pipeline.run(people_2, table_name="new_persons", write_disposition="merge", primary_key="id")



In [34]:
print('Loaded Tables:')
display(conn.sql("show tables"))

persons = conn.sql("SELECT * FROM new_persons").df()
print
total = persons['age'].sum()
print(total)

Loaded Tables:


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ new_persons         │
│ persons             │
└─────────────────────┘

266


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX